In [10]:
import datetime
import os
import random
import pandas as pd
import numpy as np
import operator
import random
from tqdm import tqdm

try:
    import binutil  # required to import from dreamcoder modules
except ModuleNotFoundError:
    import bin.binutil  # alt import if called as module

from dreamcoder.task import Task
from dreamcoder.dreamcoder import *
from dreamcoder.domains.arc.primitives import basePrimitives, tmap, taction, idx_to_action, tdirection
from dreamcoder.grammar import Grammar
from dreamcoder.utilities import testTrainSplit, eprint
from dreamcoder.type import arrow
from dreamcoder.domains.minigrid.nn_model_maze import *
from dreamcoder.dreamcoder import commandlineArguments
from dreamcoder.utilities import numberOfCPUs


In [6]:
!nvidia-smi

/usr/bin/bash: /opt/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)
Sun Feb  5 17:29:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:0A:00.0 Off |                  Off |
| 53%   82C    P2   272W / 300W |  48682MiB / 48685MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [5]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration, AutoTokenizer, TrainingArguments, Seq2SeqTrainer

target_tokenizer = AutoTokenizer.from_pretrained('google/byt5-base')
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')
#model = T5ForConditionalGeneration.from_pretrained('google/byt5-base')

In [6]:
def get_out_string(out):
    if out == 0:
        return 'zero'
    elif out == 1:
        return 'one'
    elif out == 2:
        return 'two'
    elif out == 3:
        return 'three'
    elif out == 4:
        return 'four'
    elif out == 5:
        return 'five'
    elif out == 6:
        return 'six'
    elif out == 7:
        return 'seven'
    elif out == 8:
        return 'eight'
    elif out == 9:
        return 'nine'
        
def all_equal(lst):
    return not lst or lst.count(lst[0]) == len(lst)


def parseData(taskData, groupby='run_id', verbose=False):
    columns = ['process', 'obs', 'obs direction', 
               'action', 'reward', 'done', 'run_id']

    df = pd.DataFrame(taskData, columns=columns)
    df = df.drop(['process'], axis=1)
    df.action = df.action.apply(lambda x: x[0])
    df.insert(0, 'run_id', df.pop('run_id'))

    group = df.groupby('run_id')

    groups_to_consider = []

    for key in group.groups.keys():
        g = group.get_group(key)
        if verbose:
            print(f'group {key}')
        if not g[g.reward > 0.0].count().all():
            if verbose:
                print(f'no reward..')
        else:
            reward = g[g.reward > 0.0].reward.iloc[0]
            if reward < 0.9:
                if verbose:
                    print(f'skip {key} because reward is to small')
                continue
            if verbose:
                print(f'needed {g.shape[0]} steps. Reward: {reward}')
            groups_to_consider.append(key)
    group = group.filter(lambda x: x.run_id.mean() in groups_to_consider)
    print(group.shape)
    return group.groupby(groupby)


def makeTasks(data):
    keys = data.groups.keys()
    print('keys:', len(keys))
    tasks = []
    for key in keys:
        to_imitate = data.get_group(key)
        chunkSize = random.randint(5, 25)
        examples = []
        part = 0
        for _, row in to_imitate.iterrows():
            input_ex = ((row.obs[0] * 10).astype(int).tolist(), int(row['obs direction'],))
            output_ex = int(row.action)
            examples.append((input_ex, output_ex))

            if chunkSize > 0 and chunkSize <= len(examples):
                # we check that the chosen actions are not all the same
                # otherwise it is too easy to find a program if all actions/output examples are the same
                # this results in programs such as (lambda (lambda forward-action))
                all_chosen_actions = list(zip(*examples))[1]
                if not all_equal(all_chosen_actions):
                    tasks.append(Task(f'perfect maze {key} size {chunkSize} part {part}', arrow(tmap, tdirection, taction), examples))
                    part += 1
                    # we reset examples and add new chunkSize taskss
                    examples = []
                    chunkSize = random.randint(5, 20)
                    
        if len(examples) > 3:
            all_chosen_actions = list(zip(*examples))[1]
            if not all_equal(all_chosen_actions):
                tasks.append(Task(f'perfect maze {key} size {chunkSize} part {part}', arrow(tmap, tdirection, taction), examples))

    # select random obs and actions to test
    for key in keys:
        chunkSize = random.randint(5, 25)
        to_imitate = data.get_group(key)
        examples = []
        part = 0
        already_sampled = []
        while len(to_imitate.index) - len(already_sampled) > chunkSize:
            curr_sample = random.sample([x for x in to_imitate.index if x not in already_sampled], chunkSize)
            for i in curr_sample:
                row = to_imitate.loc[i]
                input_ex = ((row.obs[0] * 10).astype(int).tolist(), int(row['obs direction'],))
                output_ex = int(row.action)
                examples.append((input_ex, output_ex))

                if chunkSize > 0 and chunkSize == len(examples):
                    # we check that the chosen actions are not all the same
                    # otherwise it is too easy to find a program if all actions/output examples are the same
                    # this results in programs such as (lambda (lambda forward-action))
                    all_chosen_actions = list(zip(*examples))[1]
                    if not all_equal(all_chosen_actions):
                        tasks.append(Task(f'perfect maze {key} size {chunkSize} random {part}', arrow(tmap, tdirection, taction), examples))
                        part += 1
                        # we reset examples and add new chunkSize taskss
                        examples = []

            already_sampled += curr_sample

    print(f'Created {len(tasks)} tasks with random chunkSize')
    return tasks


def run_dreamcoder(arguments, taskData, outputDirectory, chunkSize, resumeIteration=None, iterations=None):
    random.seed(42)
    tasks = makeTasks(taskData, chunkSize=chunkSize)
    # return tasks
    eprint("Got %d tasks..." % len(tasks))

    if len(tasks) == 0:
        return None

    arguments.pop('primitives', None)
    arguments.pop('resume', None)
    arguments.pop('iterations', None)
    # Create grammar
    grammar = Grammar.uniform(basePrimitives())

    # EC iterate
    generator = ecIteratorT5(grammar,
                           tasks,
                           outputPrefix="%s/maze" % outputDirectory,
                           resume=resumeIteration,
                           iterations=iterations,
                           **arguments)
    for i, _ in enumerate(generator):
        pass
        #print('ecIterator count {}'.format(i))

    if resumeIteration is None:
        return 1

    return int(resumeIteration) + 1


In [11]:
class DatasetCreator():
    def __init__(self, featureExtractor, grammar):
        super().__init__()
        self.id = id
        self.featureExtractor = featureExtractor
        self.generativeModel = grammar

    def taskEmbeddings(self, tasks):
        return {task: self.featureExtractor.featuresOfTask(task).data.cpu().numpy()
                for task in tasks}


    def replaceProgramsWithLikelihoodSummaries(self, frontier):
        return Frontier(
            [FrontierEntry(
                program=self.grammar.closedLikelihoodSummary(frontier.task.request, e.program),
                logLikelihood=e.logLikelihood,
                logPrior=e.logPrior) for e in frontier],
            task=frontier.task)

    def sampleHelmholtz(self, requests, statusUpdate=None, seed=None):
        if seed is not None:
            random.seed(seed)
        request = random.choice(requests)

        program = self.generativeModel.sample(request, maximumDepth=6, maxAttempts=100)
        if program is None:
            return None
        task = self.featureExtractor.taskOfProgram(program, request)

        if statusUpdate is not None:
            flushEverything()
        if task is None:
            return None

        if hasattr(self.featureExtractor, 'lexicon'):
            if self.featureExtractor.tokenize(task.examples) is None:
                return None

        ll = self.generativeModel.logLikelihood(request, program)
        frontier = Frontier([FrontierEntry(program=program,
                                           logLikelihood=0., logPrior=ll)],
                            task=task)
        return frontier
    
    def sampleProgramWithTask(self, requests, statusUpdate=None, seed=None):
        if seed is not None:
            random.seed(seed)
        request = random.choice(requests)

        program = self.generativeModel.sample(request, maximumDepth=6, maxAttempts=100)
        if program is None:
            return None
        task = self.featureExtractor.taskOfProgram(program, request)

        if statusUpdate is not None:
            flushEverything()
        if task is None:
            return None, None

        return task, program
    
    def sampleManyProgramsWithTasks(self, tasks, N):
        eprint("Sampling %d programs from the prior..." % N)
        flushEverything()
        requests = list({t.request for t in tasks})

        frequency = N / 50
        startingSeed = random.random()

        # Sequentially for ensemble training.
        data = [self.sampleProgramWithTask(requests,
                                        statusUpdate='.' if n % frequency == 0 else None,
                                        seed=startingSeed + n) for n in tqdm(range(N))]


        flushEverything()
        data = [z for z in data if not any(x is None for x in z)]
        eprint()
        eprint("Got %d/%d valid datapoints." % (len(data), N))
        flushEverything()
        
        return data
    

    def sampleManyFrontiers(self, tasks, N):
        eprint("Sampling %d programs from the prior..." % N)
        flushEverything()
        requests = list({t.request for t in tasks})

        frequency = N / 50
        startingSeed = random.random()

        # Sequentially for ensemble training.
        frontiers = [self.sampleHelmholtz(requests,
                                        statusUpdate='.' if n % frequency == 0 else None,
                                        seed=startingSeed + n) for n in range(N)]

        eprint()
        flushEverything()
        frontiers = [z for z in frontiers if z is not None]
        eprint()
        eprint("Got %d/%d valid frontiers." % (len(frontiers), N))
        flushEverything()
        
        return frontiers
    
    def createDataset(self, tasks, N, with_tasks=False):
        dataset = []
        data = self.sampleManyProgramsWithTasks(tasks, N)
        for task, program in data:
            inp_prompt = ''
            for examples in task.examples:
                inp = str(examples[0])
                out = examples[1]
                inp_string = inp.replace(',', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '')
                out_string = get_out_string(out)
                inp_prompt += f'{inp_string} {out_string} '
                
            if with_tasks:
                dataset.append((inp_prompt.strip(), str(program), task))
            else:
                dataset.append((inp_prompt.strip(), str(program)))
            
        return dataset
        
class Collator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        # entry[1] is the spec, need to call repr to turn it into a string. entry[0] is the prog_str already
        ret = {"input_ids": self.tokenizer([entry[0] for entry in batch], padding=True, return_tensors='pt').input_ids, 
            "labels": self.tokenizer([entry[1] for entry in batch], padding=True, return_tensors='pt').input_ids}
        return ret              

In [15]:
data_file = "/home/ma/e/eberhardinger/workspaces/ec/dreamcoder/domains/perfect-maze-minigrid/collected_data/2022-12-10T15:32:12.354349.npy"
data = np.load(data_file, allow_pickle=True)
parsed_data = parseData(data)
tasks = makeTasks(parsed_data)

dataset_creator = DatasetCreator(MinigridMazeFeatureExtractor(tasks), Grammar.uniform(basePrimitives()))
dataset = dataset_creator.createDataset(tasks, 50000)
np.save('gen_data_50000.npy', np.array(dataset))

(775, 6)
keys: 11
Created 109 tasks with random chunkSize
x input dim 5 y input dim 5 channel input dim 1


Sampling 50000 programs from the prior...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [10:14<00:00, 81.36it/s]

Got 50000/50000 valid datapoints.


In [20]:
training_args = TrainingArguments(per_device_train_batch_size=16, 
                                  gradient_accumulation_steps=1,
                                  save_steps = 10000,
                                  num_train_epochs=20,
                                  output_dir = './t5-checkpoints/')

from torch.utils.data import Dataset, DataLoader

class FactoringDataset(Dataset):
    def __init__(self, dataset_itself):
        self.data = dataset_itself
        
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

dataset = FactoringDataset(np.load('gen_data_50000.npy'))

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [21]:
trainer = Seq2SeqTrainer(model=model, args=training_args, train_dataset=dataset, tokenizer=tokenizer, compute_metrics=None, data_collator=Collator(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 50000
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 62500
  Number of trainable parameters = 60492288


Step,Training Loss
500,0.298100
1000,0.174800
1500,0.166400
2000,0.160700
2500,0.159400
3000,0.156600
3500,0.152000
4000,0.151700
4500,0.151200
5000,0.150700


Saving model checkpoint to ./t5-checkpoints/checkpoint-10000
Configuration saved in ./t5-checkpoints/checkpoint-10000/config.json
Configuration saved in ./t5-checkpoints/checkpoint-10000/generation_config.json
Model weights saved in ./t5-checkpoints/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in ./t5-checkpoints/checkpoint-10000/tokenizer_config.json
Special tokens file saved in ./t5-checkpoints/checkpoint-10000/special_tokens_map.json
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current va

TrainOutput(global_step=62500, training_loss=0.13949431921386718, metrics={'train_runtime': 14112.566, 'train_samples_per_second': 70.859, 'train_steps_per_second': 4.429, 'total_flos': 1.0497030185916826e+17, 'train_loss': 0.13949431921386718, 'epoch': 20.0})

In [22]:
def createTestDataFromTasks(tasks, with_tasks=False):
    dataset = []
    for task in tasks:
        inp_prompt = ''
        for examples in task.examples:
            inp = str(examples[0])
            out = examples[1]
            inp_string = inp.replace(',', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '')
            out_string = get_out_string(out)
            inp_prompt += f'{inp_string} {out_string} '

        if with_tasks:
            dataset.append((inp_prompt.strip(), task))
        else:
            dataset.append((inp_prompt.strip()))

    return dataset

# generate n samples with t temperature
def generate_samples_with_temp(txt, n_samples, temp):
    to_tokenizer = [txt for i in range(n_samples)]
    outputs = model.generate(tokenizer(to_tokenizer, return_tensors='pt', padding=True).input_ids.to('cuda'), do_sample=True, max_length=128, temperature = temp)
    results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return results

In [23]:
testTasks = createTestDataFromTasks(tasks, True)

In [24]:
generate_samples_with_temp(testTasks[0][0], 5, 1.0)

Token indices sequence length is longer than the specified maximum sequence length for this model (677 > 512). Running this sequence through the model will result in indexing errors
Generate config GenerationConfig {
  "bos_token_id": 1,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



['(lambda (lambda (if (and (and (not (eq-obj? wall-obj empty-obj)) (eq-obj? empty-obj wall-obj)) (eq-obj? (get $1 3 2) (get $1 3 0))) forward-action left-action)))',
 '(lambda (lambda (if (eq-obj? (get $1 0 3) empty-obj) left-action forward-action)))',
 '(lambda (lambda (if (eq-obj? empty-obj (get $1 3 2)) forward-action left-action)))',
 '(lambda (lambda (if (eq-direction? $0 direction-0) left-action forward-action)))',
 '(lambda (lambda (if (not (eq-obj? (get $1 2 3) empty-obj)) left-action forward-action)))']

In [25]:
testTasks[0][1]

Task(name=perfect maze 10 size 25 part 0, request=array(array(tobj)) -> tdirection -> taction, examples=[(([[2, 2, 2, 2, 2], [2, 2, 2, 2, 2], [1, 1, 1, 1, 1], [2, 2, 1, 2, 2], [1, 2, 1, 1, 1]], 0), 2), (([[2, 2, 2, 2, 2], [2, 2, 2, 2, 2], [2, 1, 1, 1, 1], [2, 2, 2, 1, 2], [2, 1, 2, 1, 1]], 0), 2), (([[2, 2, 2, 2, 2], [2, 2, 2, 2, 2], [1, 2, 1, 1, 1], [1, 2, 2, 2, 1], [1, 2, 1, 2, 1]], 0), 2), (([[2, 2, 2, 2, 2], [2, 2, 2, 2, 2], [1, 1, 2, 1, 1], [2, 1, 2, 2, 2], [2, 1, 2, 1, 2]], 0), 2), (([[2, 2, 2, 2, 2], [2, 2, 2, 2, 2], [1, 1, 1, 2, 1], [1, 2, 1, 2, 2], [1, 2, 1, 2, 1]], 0), 0), (([[2, 2, 2, 2, 1], [2, 2, 2, 2, 1], [2, 2, 2, 2, 1], [2, 2, 2, 2, 2], [2, 2, 2, 2, 1]], 3), 0), (([[1, 1, 1, 2, 1], [2, 2, 1, 2, 2], [1, 1, 1, 1, 1], [2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], 2), 2), (([[2, 1, 1, 1, 2], [2, 2, 2, 1, 2], [2, 1, 1, 1, 1], [2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], 2), 2), (([[2, 2, 1, 1, 1], [2, 2, 2, 2, 1], [2, 2, 1, 1, 1], [2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], 2), 0), (([[1, 1, 1, 2, 1], [1

In [27]:
from dreamcoder.program import Program

p = Program.parse('(lambda (lambda (if (eq-obj? (get $1 3 0) wall-obj) left-action right-action)))')
#feature_extractor.run_program_on_env(p)
p

(lambda (lambda (if (eq-obj? (get $1 3 0) wall-obj) left-action right-action)))

In [28]:
def run_on_input_examples(task, program, verbose=False):
    for inp, out in task.examples:
        # output ex is the action
        pred = runWithTimeout(lambda: program.runWithArguments(inp), None)
        if verbose:
            print('Input:', inp, 'Out:', out, 'Pred:', pred)
        if out != pred:
            return False
    return True
        
def test_programs_on_task(task, n=5, temp=1.0, verbose=False):
    progs = generate_samples_with_temp(task[0], n, temp)
    for i, prog in enumerate(progs):
        if verbose:
            print(prog)
        try:
            p = Program.parse(prog)
            
            if run_on_input_examples(task[1], p, verbose=verbose):
                return p, i
            
        except Exception as e: 
            if verbose:
                print(e)
            continue
            
    return None, n

In [30]:
from tqdm import tqdm
import transformers
transformers.logging.set_verbosity_error()
stats = []
for tt in tqdm(testTasks):       
    p, n = test_programs_on_task(tt, n=100, verbose=False)
    stats.append((p,n))

  1%|█▉                                                                                                                                                                                                                  | 1/109 [00:05<10:25,  5.79s/it]

(lambda (lambda (if (eq-obj? empty-obj (if (and (eq-direction? direction-1 direction-2)) (eq-obj? empty-obj wall-obj)) (get $1 0 3) empty-obj)) forward-action left-action)))
(lambda (lambda (if (eq-obj? (if (and (eq-obj? empty-obj wall-obj) (eq-obj? wall-obj empty-obj)) (get $1 0 3) wall-obj) (if (eq-obj? wall-obj empty-obj) (get $1 1 0) empty-obj))) left-action forward-action)))


  2%|███▉                                                                                                                                                                                                                | 2/109 [00:09<08:01,  4.50s/it]

(lambda (lambda (if (eq-direction? $0 direction-3) (if (eq-direction? $0 $0) empty-obj) forward-action left-action) left-action)))


  3%|█████▊                                                                                                                                                                                                              | 3/109 [00:12<07:04,  4.01s/it]

...

string index out of range
'int' object is not callable
(['lambda', ['lambda', ['if', ['eq-direction?', '$0', 'direction-', '0'], 'left-action', 'forward-action']]], 'direction-')
string index out of range
(lambda (lambda (if (eq-obj? empty-obj (get $1 2 2) empty-obj)) (if (eq-obj? (get (if (eq-direction? direction-1 direction-2) $1 $1) (if (eq-obj? wall-obj empty-obj) 3 1) 2) left-action) forward-action) left-action)))


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 108/109 [07:04<00:04,  4.15s/it]

(lambda (lambda (if (eq-obj? (if (eq-obj? (get $1 0 2) (get $1 2 2)) (if (eq-direction? direction-3 $0) wall-obj (get $1 2 2)) (get $1 1 1))) wall-obj) left-action (if (eq-direction? direction-0 direction-2) forward-action left-action))))
(lambda (lambda (if (eq-obj? (get $1 0 (if (eq-direction? $0 direction-1) 4 4)) (if (eq-obj? empty-obj wall-obj) (get $1 1 0) (get $1 2 2)) (get $1 3 4))) right-action forward-action)))
list indices must be integers or slices, not list
'int' object is not callable


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [07:09<00:00,  3.94s/it]

string index out of range
'int' object is not callable
(lambda (lambda (if (not (eq-obj? wall-obj wall-obj)) right-action (if (eq-obj? (if (eq-obj? empty-obj wall-obj) (get $1 1 0) (get $1 1 2)) wall-obj) (if (eq-direction? direction-1 direction-0) (if (eq-direction? direction-1 direction-1) (get $1 0 2) (get $1 3 2))) forward-action forward-action) right-action))))


In [31]:
solved = [x for x in stats if x[0] is not None]

In [32]:
print(f'{len(solved)}/{len(stats)}')
len(solved)/len(stats)

29/109


0.26605504587155965

In [33]:
solved

[((lambda (lambda (if (eq-obj? (get $1 2 3) empty-obj) forward-action left-action))),
  41),
 ((lambda (lambda (if (eq-direction? direction-0 $0) forward-action left-action))),
  16),
 ((lambda (lambda (if (not (not (eq-obj? (get $1 3 2) (get $1 3 3)))) left-action (if (eq-direction? direction-1 direction-2) forward-action forward-action)))),
  73),
 ((lambda (lambda (if (eq-obj? wall-obj (if (eq-obj? wall-obj (get $1 1 4)) (get $1 2 3) wall-obj)) left-action forward-action))),
  16),
 ((lambda (lambda (if (eq-obj? empty-obj (get $1 (if (eq-obj? empty-obj wall-obj) 3 2) 3)) forward-action (if (eq-direction? direction-0 direction-1) forward-action left-action)))),
  32),
 ((lambda (lambda (if (not (eq-direction? direction-1 $0)) left-action forward-action))),
  0),
 ((lambda (lambda (if (eq-direction? direction-1 $0) forward-action left-action))),
  0),
 ((lambda (lambda (if (not (and (eq-obj? empty-obj (get $1 2 2)) (eq-obj? wall-obj wall-obj))) left-action forward-action))),
  1),
 ((